# Import Libraries

In [28]:
import numpy as np
import ee 
# from ee_plugin import Map 
import geemap
import pprint as pprint
from datetime import datetime


In [29]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

# Import features collections and data

In [30]:
Map = geemap.Map(center=(40, -100), zoom=4)
geometry = ee.FeatureCollection('users/2014ee070/ImperialCountyBoundary')
data = ee.FeatureCollection('users/2014ee070/imperialCDWR2016')
imperial = geometry

# Defining inputs

In [45]:
start_date = '2016-01-01'
end_date = '2016-12-30'
nClasses = 28
num_classes = 28

In [46]:
classes = ["Citrus",  "Dates",  "Miscellaneous Subtropical Fruits",  "Olives",  "Miscellaneous Deciduous",  
           "Corn, Sorghum and Sudan",  "Miscellaneous Field Crops",  "Sunflowers",  
           "Miscellaneous Grain and Hay",  "Wheat",  "Managed Wetland",  "Alfalfa and Alfalfa Mixtures",  
           "Miscellaneous Grasses",  "Mixed Pasture",  "Carrots",  "Cole Crops",  
           "Flowers, Nursery and Christmas Tree Farms",  "Greenhouse",  "Lettuce/Leafy Greens",
           "Melons, Squash and Cucumbers",  "Miscellaneous Truck Crops",  "Onions and Garlic",  
           "Peppers",  "Potatoes and Sweet Potatoes",  "Tomatoes",  "Urban",  "Idle",  "Young Perennials"]


# Defining Functions

In [47]:
def maskS2sr(image):
    qa = image.select('QA60');
    cloudBitMask = 1 << 10;
    cirrusBitMask = 1 << 11;
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0));
    # Return the masked image, scaled to reflectance, without the QA bands.
    return image.updateMask(mask) \
        .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'])\
        .copyProperties(image, ["system:time_start"]);


def addNDVI(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi)


def addGNDVI(image):
    gndvi = image.normalizedDifference(['B8', 'B3']).rename('GNDVI')
    return image.addBands(gndvi)


def addEVI(image):
    evi = image.expression("2.5 * ((nir-red)/(nir + 6*red -7.5*blue + 1))",
    {
    'nir' : image.select("B8"),
    'red' : image.select("B4"),
    'blue': image.select("B2")
    }).rename('EVI')
    return image.addBands(evi)


def addSAVI(image):
    savi = image.expression("1.5*(red-green)/(red+green+0.5)",
    {
    'red' : image.select("B4"),
    'green': image.select("B3")
    }).rename('SAVI')
    return image.addBands(savi)


def addNDWI(image):
    ndwi = image.normalizedDifference(['B3', 'B8']).rename('NDWI')
    return image.addBands(ndwi)

def addNDBI(image):
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
    return image.addBands(ndbi)

def addBSI(image):
    bsi = image.expression("((swir + red) - (nir + blue))/((swir + red) + (nir + blue))",
    {
    'blue' : image.select("B2"),
    'red' : image.select("B4"),
    'nir' : image.select("B8"),
    'swir': image.select("B11")
    }).rename('BSI')
    return image.addBands(bsi) 

def ymdList(imgcol):
    def iter_func(image, newlist):
        date = ee.String(image.date().format("YYYY-MM-dd"))
        newlist = ee.List(newlist)
        return ee.List(newlist.add(date).sort())

    return imgcol.iterate(iter_func, ee.List([]))

def temporalCollection(collection, start, count, interval, units):
    sequence = ee.List.sequence(0, ee.Number(count).subtract(1))
    originalStartDate = ee.Date(start)

    def func_aoa(i):
        startDate = originalStartDate.advance(ee.Number(interval).multiply(i), units)
        endDate = originalStartDate.advance(ee.Number(interval).multiply(ee.Number(i).add(1)), units)
        resultImage = collection.filterDate(startDate, endDate).median() \
                          .set('system:time_start', startDate.millis()) \
                          .set('system:time_end', endDate.millis())
        return resultImage

    return ee.ImageCollection(sequence.map(func_aoa))


# Preprocessing Imagery Landsat 8

In [48]:
Map.centerObject(geometry)
s2Collection = ee.ImageCollection("COPERNICUS/S2") \
                  .filterDate(start_date, end_date) \
                  .filterBounds(geometry) \
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10)) \
                  .map(maskS2sr) \
                  .map(addNDVI).map(addGNDVI).map(addEVI).map(addSAVI).map(addBSI).map(addNDWI).map(addNDBI) \
                  .sort('system:time_start', True);
print("images In collection",s2Collection.size().getInfo())
s2Collection = temporalCollection(s2Collection, start_date, 12, 30, 'day')

images In collection 79


## Displaying all images selected each month

In [49]:
# # #*************** Get 12 images from s2 ***********************#

s2ImagesList = s2Collection.toList(s2Collection.size())
finalImage = ee.Image(s2ImagesList.get(0)).clip(geometry)
ts = ee.Image(s2ImagesList.get(0)).get('system:time_start').getInfo()
print(datetime.utcfromtimestamp(ts/1000).strftime('%Y-%m-%d %H:%M:%S'))

# Map.addLayer(finalImage, {'bands': ['B4', 'B3', 'B2'], 'min':0,'max': 3000}, 's2-1')

n = 1
i = 1
while i < 12:
    n = n + 1
    current = ee.Image(s2ImagesList.get(i)).clip(geometry)

    ts = current.get('system:time_start').getInfo()
    print(datetime.utcfromtimestamp(ts/1000).strftime('%Y-%m-%d %H:%M:%S'))
    
    finalImage = finalImage.addBands(current)
    # Map.addLayer(current, {'bands': ['B4', 'B3', 'B2'], 'min':0,'max': 3000}, 's2-'+n)
    i = i + 1

image = finalImage.clip(geometry).toFloat()
featuresNames = image.bandNames()
print('All features', featuresNames.getInfo())


2016-01-01 00:00:00
2016-01-31 00:00:00
2016-03-01 00:00:00
2016-03-31 00:00:00
2016-04-30 00:00:00
2016-05-30 00:00:00
2016-06-29 00:00:00
2016-07-29 00:00:00
2016-08-28 00:00:00
2016-09-27 00:00:00
2016-10-27 00:00:00
2016-11-26 00:00:00
All features ['B2', 'B3', 'B4', 'B8', 'B11', 'B12', 'NDVI', 'GNDVI', 'EVI', 'SAVI', 'BSI', 'NDWI', 'NDBI', 'B2_1', 'B3_1', 'B4_1', 'B8_1', 'B11_1', 'B12_1', 'NDVI_1', 'GNDVI_1', 'EVI_1', 'SAVI_1', 'BSI_1', 'NDWI_1', 'NDBI_1', 'B2_2', 'B3_2', 'B4_2', 'B8_2', 'B11_2', 'B12_2', 'NDVI_2', 'GNDVI_2', 'EVI_2', 'SAVI_2', 'BSI_2', 'NDWI_2', 'NDBI_2', 'B2_3', 'B3_3', 'B4_3', 'B8_3', 'B11_3', 'B12_3', 'NDVI_3', 'GNDVI_3', 'EVI_3', 'SAVI_3', 'BSI_3', 'NDWI_3', 'NDBI_3', 'B2_4', 'B3_4', 'B4_4', 'B8_4', 'B11_4', 'B12_4', 'NDVI_4', 'GNDVI_4', 'EVI_4', 'SAVI_4', 'BSI_4', 'NDWI_4', 'NDBI_4', 'B2_5', 'B3_5', 'B4_5', 'B8_5', 'B11_5', 'B12_5', 'NDVI_5', 'GNDVI_5', 'EVI_5', 'SAVI_5', 'BSI_5', 'NDWI_5', 'NDBI_5', 'B2_6', 'B3_6', 'B4_6', 'B8_6', 'B11_6', 'B12_6', 'NDVI_6'

# FIltering and remapping CRS data labels

In [50]:
#************* remapping crops to major crops *****************#

classSizes = {}
for i in range(0, nClasses, 1):
    classSizes[classes[i]] = data.filter(ee.Filter.eq('landover', i)).size().getInfo()

pprint.pprint(classes)
pprint.pprint(classSizes)

['Citrus',
 'Dates',
 'Miscellaneous Subtropical Fruits',
 'Olives',
 'Miscellaneous Deciduous',
 'Corn, Sorghum and Sudan',
 'Miscellaneous Field Crops',
 'Sunflowers',
 'Miscellaneous Grain and Hay',
 'Wheat',
 'Managed Wetland',
 'Alfalfa and Alfalfa Mixtures',
 'Miscellaneous Grasses',
 'Mixed Pasture',
 'Carrots',
 'Cole Crops',
 'Flowers, Nursery and Christmas Tree Farms',
 'Greenhouse',
 'Lettuce/Leafy Greens',
 'Melons, Squash and Cucumbers',
 'Miscellaneous Truck Crops',
 'Onions and Garlic',
 'Peppers',
 'Potatoes and Sweet Potatoes',
 'Tomatoes',
 'Urban',
 'Idle',
 'Young Perennials']
{'Alfalfa and Alfalfa Mixtures': 2156,
 'Carrots': 212,
 'Citrus': 215,
 'Cole Crops': 277,
 'Corn, Sorghum and Sudan': 232,
 'Dates': 34,
 'Flowers, Nursery and Christmas Tree Farms': 10,
 'Greenhouse': 12,
 'Idle': 865,
 'Lettuce/Leafy Greens': 619,
 'Managed Wetland': 22,
 'Melons, Squash and Cucumbers': 153,
 'Miscellaneous Deciduous': 2,
 'Miscellaneous Field Crops': 187,
 'Miscellaneous 

## Splitting data and making image

In [51]:
imperial_data = data.randomColumn('random', 2016)
training = imperial_data.filter(ee.Filter.lt('random', 0.6));
validation = imperial_data.filter(ee.Filter.And(
                      ee.Filter.gte('random', 0.6), 
                      ee.Filter.lt('random', 0.8)));
testing = imperial_data.filter(ee.Filter.gte('random', 0.8));
print(training.size().getInfo(), validation.size().getInfo(), testing.size().getInfo())

training_label = training.reduceToImage(
    properties =  ['landover'],
    reducer = ee.Reducer.first()
)
val_label = validation.reduceToImage(
    properties =  ['landover'],
    reducer = ee.Reducer.first()
)
testing_label = testing.reduceToImage(
    properties =  ['landover'],
    reducer = ee.Reducer.first()
)



4373 1395 1498


In [52]:
Map.addLayer(training_label, {'palette': 'FF0000'}, 'Training')
Map.addLayer(val_label, {'palette': '0000FF'}, 'validation')
Map.addLayer(testing_label, {'palette': '0000FF'}, 'Testing')
Map.centerObject(training, 10)
Map

Map(bottom=105919.0, center=[32.93580026550981, -115.5107517261226], controls=(WidgetControl(options=['positio…

In [53]:
label = imperial_data.reduceToImage(
    properties =  ['landover'],
    reducer = ee.Reducer.first()
)
label = label.select('first').rename('landcover')


## Making final Image

In [54]:
kernel_size = 3
image = finalImage.addBands(label).toFloat()

neighbourhood_image = image.neighborhoodToArray(ee.Kernel.rectangle(kernel_size, kernel_size, 'pixels'))
Map.addLayer(neighbourhood_image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}, 's2_extended')
Map.centerObject(imperial)
Map

Map(bottom=105919.0, center=[32.93979228798953, -115.5223403258469], controls=(WidgetControl(options=['positio…

In [55]:
neighbourhood_image.bandNames().getInfo()

['B2',
 'B3',
 'B4',
 'B8',
 'B11',
 'B12',
 'NDVI',
 'GNDVI',
 'EVI',
 'SAVI',
 'BSI',
 'NDWI',
 'NDBI',
 'B2_1',
 'B3_1',
 'B4_1',
 'B8_1',
 'B11_1',
 'B12_1',
 'NDVI_1',
 'GNDVI_1',
 'EVI_1',
 'SAVI_1',
 'BSI_1',
 'NDWI_1',
 'NDBI_1',
 'B2_2',
 'B3_2',
 'B4_2',
 'B8_2',
 'B11_2',
 'B12_2',
 'NDVI_2',
 'GNDVI_2',
 'EVI_2',
 'SAVI_2',
 'BSI_2',
 'NDWI_2',
 'NDBI_2',
 'B2_3',
 'B3_3',
 'B4_3',
 'B8_3',
 'B11_3',
 'B12_3',
 'NDVI_3',
 'GNDVI_3',
 'EVI_3',
 'SAVI_3',
 'BSI_3',
 'NDWI_3',
 'NDBI_3',
 'B2_4',
 'B3_4',
 'B4_4',
 'B8_4',
 'B11_4',
 'B12_4',
 'NDVI_4',
 'GNDVI_4',
 'EVI_4',
 'SAVI_4',
 'BSI_4',
 'NDWI_4',
 'NDBI_4',
 'B2_5',
 'B3_5',
 'B4_5',
 'B8_5',
 'B11_5',
 'B12_5',
 'NDVI_5',
 'GNDVI_5',
 'EVI_5',
 'SAVI_5',
 'BSI_5',
 'NDWI_5',
 'NDBI_5',
 'B2_6',
 'B3_6',
 'B4_6',
 'B8_6',
 'B11_6',
 'B12_6',
 'NDVI_6',
 'GNDVI_6',
 'EVI_6',
 'SAVI_6',
 'BSI_6',
 'NDWI_6',
 'NDBI_6',
 'B2_7',
 'B3_7',
 'B4_7',
 'B8_7',
 'B11_7',
 'B12_7',
 'NDVI_7',
 'GNDVI_7',
 'EVI_7',
 'SAVI_7',
 '

# Sampling

In [56]:
# sampling only points for now
training_points = image.select('landcover')\
          .sampleRegions(
            collection= training,
            scale=80,
            tileScale =1,
            properties= ['landcover'],
            geometries = True
          )
val_points = image.select('landcover')\
          .sampleRegions(
            collection= validation,
            scale=80,
            tileScale = 1,
            properties= ['landcover'],
            geometries = True
          )

testing_points = image.select('landcover')\
          .sampleRegions(
            collection= testing,
            scale=80,
            tileScale = 1,
            properties= ['landcover'],
            geometries = True
          )


In [16]:
# get lat longs of sampled points
training_pts = training_points.geometry().getInfo()['coordinates']
val_pts = val_points.geometry().getInfo()['coordinates']
testing_pts = testing_points.geometry().getInfo()['coordinates']


In [17]:
testing_points.first().getInfo()

{'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'Point',
  'coordinates': [-115.73067670534525, 33.08279595744009]},
 'id': '00000000000000001073_0',
 'properties': {}}

In [18]:
len(training_pts), len(val_pts), len(testing_pts)

(209767, 64907, 70157)

In [57]:
patch2 = ee.Geometry.Polygon([[-115.58345059458601,32.73187233683694],
    [-115.3465578943907,32.73187233683694],
    [-115.3465578943907,32.943604017691634],
    [-115.58345059458601,32.943604017691634],
    [-115.58345059458601,32.73187233683694]])

In [58]:
Task = ee.batch.Export.image.toDrive(
    image = image.clip(patch2),
    description = 'patch of imperial',
    folder = 'Imperial_2016_patch',
    fileNamePrefix = 'Imperial_2016_patch3',
    scale = 10,
    fileFormat = 'GeoTIFF',
    region = patch2
)
Task.start()

In [25]:
# testing_features = testing.toList(1498)
# for i in range(1498):
#     feature = ee.Feature(testing_features.get(i))
#     fcp = testing_points.filterBounds(feature.geometry())
#     train_db = neighbourhood_image.sampleRegions(collection=fcp, scale=10)
#     Task = ee.batch.Export.table.toDrive(
#         collection=train_db,        
#         description="testing_imperial_2016"+str(i),
#         fileNamePrefix="testing_imperial_2016_"+str(i),
#         folder = "testing_polygons_imperial_2016",  
#         fileFormat='TFRecord')
#     Task.start()
#     print(i)

In [26]:
# val_features = validation.toList(1395)
# for i in range(1395):
#     feature = ee.Feature(val_features.get(i))
#     fcp = val_points.filterBounds(feature.geometry())
#     train_db = neighbourhood_image.sampleRegions(collection=fcp, scale=10)
#     Task = ee.batch.Export.table.toDrive(
#         collection=train_db,        
#         description="validation_imperial_2016"+str(i),
#         fileNamePrefix="validation_imperial_2016_"+str(i),
#         folder = "validation_polygons_imperial_2016",  
#         fileFormat='TFRecord')
#     Task.start()
#     print(i)

In [29]:
filenames = []
nbands = 12*13+1
nfeatures = 7*7*13*70157 #estimate the totals # of features

nparts = int(np.ceil(nfeatures/1e3))
print('Dataset too long, splitting it into '+ str(nparts),'equal parts.')


nppoints = np.array(testing_pts)
np.random.shuffle(nppoints)

count_batch = 1  # Batch counter 

for batch_arr in np.array_split(nppoints,nparts):

    fcp = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(p),{'class':'NA'}) for p in batch_arr.tolist()])

    train_db = neighbourhood_image.sampleRegions(collection=fcp, scale=30, geometries = True)
    print(train_db.first().getInfo())
    
#     filename = file_name_prefix + str(count_batch)
#     print('sending the task #%04d'%count_batch)
#     Task = ee.batch.Export.table.toDrive(
#             collection=train_db,        
#             description=file_name_prefix+str(count_batch),
#             fileNamePrefix=filename,
#             folder = folder_name,  
#             fileFormat='TFRecord')

#     Task.start()
    break
    filenames.append(filename)
    count_batch+=1

Dataset too long, splitting it into 44691 equal parts.
{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Point', 'coordinates': [-115.72784701220029, 32.81110049975814]}, 'id': '0_0', 'properties': {'B11': [[1832, 1852, 1768, 1723, 1662, 1992, 2503], [2076, 2182, 2101, 2011, 1835, 2153, 2638], [2778, 2763, 2744, 2664, 2467, 2609, 2705], [3123, 3120, 3073, 3084, 3074, 3069, 2958], [3045, 3009, 3025, 3039, 2980, 3056, 2888], [3052, 3016, 3021, 2903, 3017, 3149, 2783], [3109, 3098, 3054, 3017, 3081, 3107, 2788]], 'B11_1': [[1364.5, 1384, 1334.5, 1301.5, 1282.5, 1728.5, 2506], [1451, 1503, 1506.5, 1463.5, 1388, 1907.5, 2631.5], [1995, 2076, 2092, 2017, 2018, 2418.5, 2785.5], [1649, 1661, 1666, 1724, 2102.5, 2826, 3064], [1498, 1495.5, 1496, 1511, 1741, 2592, 2954.5], [1504.5, 1502.5, 1507, 1503.5, 1702.5, 2562.5, 2884.5], [1529.5, 1529.5, 1524, 1525.5, 1724, 2546.5, 2857.5]], 'B11_10': [[1356, 1346, 1334, 1299, 1263, 1850, 2494], [1500, 1497, 1511, 1487, 1459, 2091, 2564], [2175

# EXPORT

In [18]:
def export_2D_samples(collection, num_of_images, num_features, dimension, file_name_prefix, folder_name):
    filenames = []
    nbands = num_of_images*num_features+1
    nfeatures = dimension*dimension*nbands*len(collection) #estimate the totals # of features

    nparts = int(np.ceil(nfeatures/3e6))
    print('Dataset too long, splitting it into '+ str(nparts),'equal parts.')


    nppoints = np.array(collection)
    np.random.shuffle(nppoints)

    count_batch = 1  # Batch counter 

    for batch_arr in np.array_split(nppoints,nparts):

        fcp = ee.FeatureCollection([ee.Feature(ee.Geometry.Point(p),{'class':'NA'}) for p in batch_arr.tolist()])

        train_db = neighbourhood_image.sampleRegions(collection=fcp, scale=10, geometries = True)

        filename = file_name_prefix + str(count_batch)
        print('sending the task #%04d'%count_batch)
        Task = ee.batch.Export.table.toDrive(
                collection=train_db,        
                description=file_name_prefix+str(count_batch),
                fileNamePrefix=filename,
                folder = folder_name,  
                fileFormat='TFRecord')

        Task.start()
        filenames.append(filename)
        count_batch+=1

In [19]:
export_2D_samples(collection = training_pts, 
                  num_of_images = 12, 
                  num_features = 13, 
                  dimension = 7, 
                  file_name_prefix = "training_imperial_2016_", 
                  folder_name = "training_imperial_2016_28c")

Dataset too long, splitting it into 538 equal parts.
sending the task #0001
sending the task #0002
sending the task #0003
sending the task #0004
sending the task #0005
sending the task #0006
sending the task #0007
sending the task #0008
sending the task #0009
sending the task #0010
sending the task #0011
sending the task #0012
sending the task #0013
sending the task #0014
sending the task #0015
sending the task #0016
sending the task #0017
sending the task #0018
sending the task #0019
sending the task #0020
sending the task #0021
sending the task #0022
sending the task #0023
sending the task #0024
sending the task #0025
sending the task #0026
sending the task #0027
sending the task #0028
sending the task #0029
sending the task #0030
sending the task #0031
sending the task #0032
sending the task #0033
sending the task #0034
sending the task #0035
sending the task #0036
sending the task #0037
sending the task #0038
sending the task #0039
sending the task #0040
sending the task #0041
send

sending the task #0355
sending the task #0356
sending the task #0357
sending the task #0358
sending the task #0359
sending the task #0360
sending the task #0361
sending the task #0362
sending the task #0363
sending the task #0364
sending the task #0365
sending the task #0366
sending the task #0367
sending the task #0368
sending the task #0369
sending the task #0370
sending the task #0371
sending the task #0372
sending the task #0373
sending the task #0374
sending the task #0375
sending the task #0376
sending the task #0377
sending the task #0378
sending the task #0379
sending the task #0380
sending the task #0381
sending the task #0382
sending the task #0383
sending the task #0384
sending the task #0385
sending the task #0386
sending the task #0387
sending the task #0388
sending the task #0389
sending the task #0390
sending the task #0391
sending the task #0392
sending the task #0393
sending the task #0394
sending the task #0395
sending the task #0396
sending the task #0397
sending the

In [20]:
export_2D_samples(collection = val_pts, 
                  num_of_images = 12, 
                  num_features = 13, 
                  dimension = 7, 
                  file_name_prefix = "validation_imperial_2016_", 
                  folder_name = "validation_imperial_2016_28c")

Dataset too long, splitting it into 167 equal parts.
sending the task #0001
sending the task #0002
sending the task #0003
sending the task #0004
sending the task #0005
sending the task #0006
sending the task #0007
sending the task #0008
sending the task #0009
sending the task #0010
sending the task #0011
sending the task #0012
sending the task #0013
sending the task #0014
sending the task #0015
sending the task #0016
sending the task #0017
sending the task #0018
sending the task #0019
sending the task #0020
sending the task #0021
sending the task #0022
sending the task #0023
sending the task #0024
sending the task #0025
sending the task #0026
sending the task #0027
sending the task #0028
sending the task #0029
sending the task #0030
sending the task #0031
sending the task #0032
sending the task #0033
sending the task #0034
sending the task #0035
sending the task #0036
sending the task #0037
sending the task #0038
sending the task #0039
sending the task #0040
sending the task #0041
send

In [21]:
export_2D_samples(collection = testing_pts, 
                  num_of_images = 12, 
                  num_features = 13, 
                  dimension = 7, 
                  file_name_prefix = "testing_imperial_2016_", 
                  folder_name = "testing_imperial_2016_28c")

Dataset too long, splitting it into 180 equal parts.
sending the task #0001
sending the task #0002
sending the task #0003
sending the task #0004
sending the task #0005
sending the task #0006
sending the task #0007
sending the task #0008
sending the task #0009
sending the task #0010
sending the task #0011
sending the task #0012
sending the task #0013
sending the task #0014
sending the task #0015
sending the task #0016
sending the task #0017
sending the task #0018
sending the task #0019
sending the task #0020
sending the task #0021
sending the task #0022
sending the task #0023
sending the task #0024
sending the task #0025
sending the task #0026
sending the task #0027
sending the task #0028
sending the task #0029
sending the task #0030
sending the task #0031
sending the task #0032
sending the task #0033
sending the task #0034
sending the task #0035
sending the task #0036
sending the task #0037
sending the task #0038
sending the task #0039
sending the task #0040
sending the task #0041
send